In [1]:
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import ipyvolume as ipv
import nibabel as nib

In [2]:
def create_3D_segmentations(filepath, downsample_rate=1):
    """
    Стандартные лейблы были:
    1 - liver
    2 - spleen
    3 - kidney_left
    4 - kidney_right
    5 - bowel
    Изменю структуру, чтобы было более удобно на такой вариант

    1 - liver
    2 - spleen
    3 - kidney (r & l)
    4 - bowel
    """
    img = nib.load(filepath).get_fdata()
    img = np.transpose(img, [1, 0, 2])
    img = np.rot90(img, 1, (1, 2))
    img = img[::-1, :, :]
    img = np.transpose(img, [1, 0, 2])
    img = img[::downsample_rate, ::downsample_rate, ::downsample_rate]
    img = np.transpose(img, [1, 2, 0])
    img = np.round(img).astype(int)

    # 4 -> 3
    img = np.where(img == 4, 3, img)
    # 5 -> 4
    img = np.where(img == 5, 4, img)

    return img

In [32]:
%%time
path_to_mask = r"C:\KAGGLE\MEDICINE\data\segmentations\51033.nii"
mask_data = create_3D_segmentations(path_to_mask)

CPU times: total: 6.39 s
Wall time: 9.48 s


In [33]:
name = 51033

In [37]:
new_path_to_save = os.path.join(
    r"C:\KAGGLE\MEDICINE\data\segmentations_numpy", str(name) + ".npy"
)
np.save(new_path_to_save, mask_data)

In [39]:
%%time
mask_data = np.load(new_path_to_save)

CPU times: total: 375 ms
Wall time: 429 ms


In [27]:
def process(data_path="", size=512):
    """
    Read img and convert in np.array with standardization data.
    :param data_path: path to folder with dcm files img
    :param size: size to return img
    :return:
    """
    lst_files = os.listdir(data_path)
    lst_files = [int(x[:-4]) for x in lst_files]
    imgs = []
    for f in range(min(lst_files), max(lst_files) + 1):
        path_to_files = os.path.join(data_path, f"{f}.dcm")

        dicom = pydicom.dcmread(path_to_files)

        img = standardize_pixel_array(dicom)
        img = (img - img.min()) / (img.max() - img.min() + 1e-6)

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            img = 1 - img

        if img.shape != (512, 512):
            img = cv2.resize(img, (size, size))

        imgs.append(img)

    combined_array = np.stack(imgs, axis=0)

    combined_array = np.transpose(combined_array, [1, 2, 0])

    return combined_array

In [28]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype
        pixel_array = (pixel_array << bit_shift).astype(dtype) >> bit_shift
    #         pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)

    intercept = float(dcm.RescaleIntercept)
    slope = float(dcm.RescaleSlope)
    center = int(dcm.WindowCenter)
    width = int(dcm.WindowWidth)
    low = center - width / 2
    high = center + width / 2

    pixel_array = (pixel_array * slope) + intercept
    pixel_array = np.clip(pixel_array, low, high)

    return pixel_array

In [30]:
%%timeit
dicom_folder = r"C:\KAGGLE\MEDICINE\data\train_images\10004\51033"
img_new = process(data_path=dicom_folder, size=512)
img = img_new

12 s ± 1.17 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
img.shape

(512, 512, 1044)

In [45]:
# Создайте трехмерную визуализацию
fig = ipv.figure()

# Отобразите объемные данные
# vol = ipv.volshow( img +  mask_data, lighting=True)
vol = ipv.volshow(img + np.where(mask_data == 1, 1, 0), lighting=True)
# Добавьте интерактивность (вращение, масштабирование и т.д.)
ipv.style.box_off()
ipv.style.axes_off()
ipv.style.set_style_light()

# Покажите визуализацию
ipv.show()

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [7]:
mask_data

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 